In [1]:
import pandas
import numpy as np
data = pandas.read_csv('train-set.csv')
train_X = data.iloc[:, 1:-1]
train_y = data.iloc[:, -1]
test_X_temp = pandas.read_csv('test-set.csv')
test_X = test_X_temp.iloc[:, 1:]

In [2]:
#The initial idea was to replace exclusive columns like Soil_Type and Wilderness_Area, but delete old columns will not
#give a better result, so newly created columns are kept.
def exclusive_comb(X_temp):
    features = X_temp
    
    features['Soil_Type'] = 0
    for i in range(1, 41):
        features['Soil_Type'] = features['Soil_Type'] + i * X_temp['Soil_Type'+str(i)]
        #features.drop('Soil_Type'+str(i),inplace=True, axis=1)
    
    features['Wilderness_Area'] = 0
    for i in range(1, 5):
        features['Wilderness_Area'] = features['Wilderness_Area'] + i * X_temp['Wilderness_Area'+str(i)]
        #features.drop('Wilderness_Area'+str(i), inplace=True, axis=1)
        
    features['Distance_To_Hydrology'] = (features['Horizontal_Distance_To_Hydrology']**2 + features['Vertical_Distance_To_Hydrology']**2)**0.5
    #features.drop(['Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology'], axis = 1, inplace = True)
    
    features['Ele_minus_VDtHyd'] = features['Elevation']-features['Vertical_Distance_To_Hydrology']
    features['Ele_plus_VDtHyd'] = features['Elevation']+features['Vertical_Distance_To_Hydrology']
    features['Hydro_plus_Fire'] = features['Horizontal_Distance_To_Hydrology']+features['Horizontal_Distance_To_Fire_Points']
    features['Hydro_minus_Fire'] = features['Horizontal_Distance_To_Hydrology']-features['Horizontal_Distance_To_Fire_Points']
    features['Hydro_plus_Road'] = features['Horizontal_Distance_To_Hydrology']+features['Horizontal_Distance_To_Roadways']
    features['Hydro_minus_Road'] = features['Horizontal_Distance_To_Hydrology']-features['Horizontal_Distance_To_Roadways']
    features['Fire_plus_Road'] = features['Horizontal_Distance_To_Fire_Points']+features['Horizontal_Distance_To_Roadways']
    features['Fire_minus_Road'] = features['Horizontal_Distance_To_Fire_Points']-features['Horizontal_Distance_To_Roadways']
    
    return features

In [3]:
train_X = exclusive_comb(train_X)
test_X = exclusive_comb(test_X)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [23]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_X, train_y)
result_y = logreg.predict(test_X)


In [7]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100, criterion='entropy')
rfc.fit(train_X, train_y)
result_y = rfc.predict(test_X)
predictions = pandas.DataFrame({'Id': test_X_temp.iloc[:,0], 'Cover_Type': result_y})

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
xbc = GradientBoostingClassifier()
xbc.fit(train_X, train_y)
result_y = xbc.predict(test_X)
predictions = pandas.DataFrame({'Id': test_X_temp.iloc[:,0], 'Cover_Type': result_y})

In [38]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(train_X, train_y)
result_y = mlp.predict(test_X)
predictions = pandas.DataFrame({'Id': test_X_temp.iloc[:,0], 'Cover_Type': result_y})

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [6]:
#In the first run of selecting classifiers with default parameters, best result is achieved by ExtraTreesClassifier
#So the cross validation is performed here
from sklearn.metrics import fbeta_score, make_scorer, accuracy_score
from sklearn.model_selection  import StratifiedShuffleSplit
from sklearn.model_selection  import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
rfc = ExtraTreesClassifier()
parameters = {'n_estimators': [ 50, 100, 200, 500 ],
                  'criterion': ['gini','entropy'],
                  #'min_samples_split': [0.1, 0.05, 0.02, 0.01], 
                  #'max_depth': [4, 6],
                  'max_features': [1.0, 0.3, 0.1]
                 }
scorer = make_scorer(fbeta_score, beta=0.5, average='weighted')
cv = StratifiedShuffleSplit(train_y, test_size=0.2, random_state=42)
grid_obj = GridSearchCV(rfc, param_grid=parameters, cv=cv, scoring=scorer, verbose=1)
grid_fit = grid_obj.fit(train_X, train_y)
rfc_best    = grid_obj.best_estimator_
rfc_best.fit(train_X, train_y)
result_y = rfc_best.predict(test_X)
predictions = pandas.DataFrame({'Id': test_X_temp.iloc[:,0], 'Cover_Type': result_y})

In [7]:
predictions.to_csv('predictions.csv', index=False)
